# Cyclone tropical Nadine (septembre 2012) : analyse et prévision déterministe

Auteur : FERRY Frédéric (ENM/C3M) - Mars 2022

Cas du cyclone tropical Nadine sur l'Alantique. Analyse et prévision déterministe du run du 20/09/2012 0000UTC.

Concepts Python illustrés :

- Exploitation de fichiers netcdf (xarray)
- Tracé de cartes (matplotlib, cartopy)
- Filtrage spatial gaussien (module gaussian_filter de scipy)
- Pointage des min/max

In [97]:
%matplotlib inline

import os

import xarray as xr
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid

from cartopy import config
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

from scipy.ndimage import gaussian_filter

import IPython.display as IPdisplay, matplotlib.font_manager as fm
from PIL import Image
import glob

In [98]:
# Analysis data
Ana_Z500 = './data/Z500_September2012_HiRes.nc'
Ana_MSLP = './data/MSLP_September2012_HiRes.nc'

# Analysis time
t0 ='2012-09-15T00:00:00.000000000'
t1 ='2012-09-20T00:00:00.000000000'

# Hires
Fcst_Z = './data/geop_fc_20120920-allsteps-gridded.nc'
Fcst_MSLP = './data/mslp_fc_20120920-allsteps-gridded.nc'

# Domain for field plots
latS = 20.
latN = 70.
lonW = -60.
lonE = 30.

In [99]:
projection=ccrs.PlateCarree()
bounds = [(lonW, lonE, latS, latN)]

def plot_background(ax):
    ax.coastlines()
    ax.gridlines()
    ax.set_xticks(np.linspace(-180, 180, 19), crs=ccrs.PlateCarree())
    ax.set_yticks(np.linspace(-90, 90, 19), crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    return(ax)             

In [100]:
def plot_maxmin_points(lon, lat, data, extrema, nsize, symbol, color='k',
                       plotValue=True, transform=None):
    """
    This function will find and plot relative maximum and minimum for a 2D grid. The function
    can be used to plot an H for maximum values (e.g., High pressure) and an L for minimum
    values (e.g., low pressue). It is best to used filetered data to obtain  a synoptic scale
    max/min value. The symbol text can be set to a string value and optionally the color of the
    symbol and any plotted value can be set with the parameter color
    lon = plotting longitude values (2D)
    lat = plotting latitude values (2D)
    data = 2D data that you wish to plot the max/min symbol placement
    extrema = Either a value of max for Maximum Values or min for Minimum Values
    nsize = Size of the grid box to filter the max and min values to plot a reasonable number
    symbol = String to be placed at location of max/min value
    color = String matplotlib colorname to plot the symbol (and numerica value, if plotted)
    plot_value = Boolean (True/False) of whether to plot the numeric value of max/min point
    The max/min symbol will be plotted on the current axes within the bounding frame
    (e.g., clip_on=True)
    """
    from scipy.ndimage.filters import maximum_filter, minimum_filter

    if (extrema == 'max'):
        data_ext = maximum_filter(data, nsize, mode='nearest')
    elif (extrema == 'min'):
        data_ext = minimum_filter(data, nsize, mode='nearest')
    else:
        raise ValueError('Value for hilo must be either max or min')

    mxy, mxx = np.where(data_ext == data)

    for i in range(len(mxy)):
        ax.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]], symbol, color=color, size=12,
                clip_on=True, horizontalalignment='center', verticalalignment='center',
                transform=transform)
        ax.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]],
                '\n' + str(int(data[mxy[i], mxx[i]])),
                color=color, size=10, clip_on=True, fontweight='bold',
                horizontalalignment='center', verticalalignment='top', transform=transform)

In [101]:
def make_animation():
    nbimages=len(time)
    # create a tuple of display durations, one for each frame
    first_last = 1000 #show the first and last frames for 100 ms
    standard_duration = 1000 #show all other frames for 5 ms
    durations = tuple([first_last] + [standard_duration] * (nbimages - 2) + [first_last])
    # load all the static images into a list
    images = [Image.open(image) for image in sorted(glob.glob('{}/*.png'.format('./anim/')))]
    # save as an animated gif
    gif = images[0]
    gif.info['duration'] = durations #ms per frame
    gif.info['loop'] = 0 #how many times to loop (0=infinite)
    gif.save(fp=gif_filepath, format='gif', save_all=True, append_images=images[1:])
    # verify that the number of frames in the gif equals the number of image files and durations
    Image.open(gif_filepath).n_frames == len(images) == len(durations)
    # clean png
    os.chdir("./anim/")
    for f in glob.glob("*.png"):
        os.remove(f)
    os.chdir("../")
    return Image

# Partie 1 : étude des données d'analyse (15-20 septembre 2012)

In [102]:
data_z    = xr.open_dataset(Ana_Z500)
data_mslp   = xr.open_dataset(Ana_MSLP)

print(data_z)
print(data_mslp)

<xarray.Dataset>
Dimensions:  (lon: 301, lat: 121, lev: 1, time: 120)
Coordinates:
  * lon      (lon) float64 -90.0 -89.5 -89.0 -88.5 -88.0 ... 58.5 59.0 59.5 60.0
  * lat      (lat) float64 80.0 79.5 79.0 78.5 78.0 ... 22.0 21.5 21.0 20.5 20.0
  * lev      (lev) float64 5e+04
  * time     (time) datetime64[ns] 2012-09-01 ... 2012-09-30T18:00:00
Data variables:
    var129   (time, lev, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version 1.4.3
    Conventions:  CF-1.0
    history:      Tue Aug 20 18:22:29 2019: cdo -f nc copy Z500_September2012...
    institution:  European Centre for Medium-Range Weather Forecasts
    CDO:          Climate Data Operators version 1.4.3 (http://www.mpimet.mpg...
<xarray.Dataset>
Dimensions:  (lon: 1067, lat: 429, time: 120)
Coordinates:
  * lon      (lon) float64 -90.14 -90.0 -89.86 -89.72 ... 59.88 60.02 60.16
  * lat      (lat) float64 80.22 80.08 79.94 79.8 ... 20.3 20.16 20.02 19.87
  * time     (time) datetime64[ns] 20

In [103]:
data_z    = xr.open_dataset(Ana_Z500).sel(lat=slice(latN,latS)).sel(lon=slice(lonW,lonE)).sel(time=slice(t0,t1)).squeeze()
data_mslp   = xr.open_dataset(Ana_MSLP).sel(lat=slice(latN,latS)).sel(lon=slice(lonW,lonE)).sel(time=slice(t0,t1)).squeeze()

print(data_z)
print(data_mslp)

lats_z = data_z.lat.values
lons_z = data_z.lon.values
lats_mslp = data_mslp.lat.values
lons_mslp = data_mslp.lon.values

z_ana = data_z['var129']/9.81
mslp_ana = data_mslp['var151']/100

time = data_mslp.time.values
times_str=[x for x in range(len(time))]
dates_str=[x for x in range(len(time))]

for i in range(len(time)):
	times_str[i] = str(time[i])
	dates_str[i] = times_str[i][0:13]

print(dates_str)

<xarray.Dataset>
Dimensions:  (lon: 181, lat: 101, time: 21)
Coordinates:
  * lon      (lon) float64 -60.0 -59.5 -59.0 -58.5 -58.0 ... 28.5 29.0 29.5 30.0
  * lat      (lat) float64 70.0 69.5 69.0 68.5 68.0 ... 22.0 21.5 21.0 20.5 20.0
    lev      float64 5e+04
  * time     (time) datetime64[ns] 2012-09-15 2012-09-15T06:00:00 ... 2012-09-20
Data variables:
    var129   (time, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version 1.4.3
    Conventions:  CF-1.0
    history:      Tue Aug 20 18:22:29 2019: cdo -f nc copy Z500_September2012...
    institution:  European Centre for Medium-Range Weather Forecasts
    CDO:          Climate Data Operators version 1.4.3 (http://www.mpimet.mpg...
<xarray.Dataset>
Dimensions:  (lon: 639, lat: 355, time: 21)
Coordinates:
  * lon      (lon) float64 -59.97 -59.83 -59.69 -59.55 ... 29.71 29.85 29.99
  * lat      (lat) float64 69.93 69.79 69.65 69.51 ... 20.44 20.3 20.16 20.02
  * time     (time) datetime64[ns] 2012-09-15 

In [104]:
z_ana_filter = gaussian_filter(z_ana, sigma=2.0)
mslp_ana_filter = gaussian_filter(mslp_ana, sigma=2.0)

lon_grid_z, lat_grid_z = np.meshgrid(lons_z, lats_z)
lon_grid_mslp, lat_grid_mslp = np.meshgrid(lons_mslp, lats_mslp)

In [105]:
print(time.shape)
cmap='jet'
clevs1 = np.linspace(5000, 6000, 21)
clevs2 = np.linspace(980, 1040, 21)

for i in range(len(time)):
    print(dates_str[i])
    fig = plt.figure(figsize=(15, 15))
    fig.suptitle('Analysis : '+dates_str[i], fontsize=14)
    ax = fig.add_subplot(2, 1, 1, projection=ccrs.PlateCarree())
    plt.title('Geopotential height at 500 hPa', size=10, loc='center')
    plot_background(ax)
    cf = ax.contourf(lons_z, lats_z, z_ana[i,:,:], levels=clevs1, cmap=cmap, extend='both',
                     transform=ccrs.PlateCarree())
    c = ax.contour(lons_z, lats_z, z_ana[i,:,:], levels=clevs1, colors='black', linewidths=0.5,
                   transform=ccrs.PlateCarree())
    ax.clabel(c, fmt='%4.1i', fontsize=10)
    plot_maxmin_points(lon_grid_z, lat_grid_z, z_ana_filter[i,:,:], 'max', 50, symbol='H', color='r',
                       transform=ccrs.PlateCarree())
    plot_maxmin_points(lon_grid_z, lat_grid_z, z_ana_filter[i,:,:], 'min', 25, symbol='L', color='b',
                       transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='vertical', aspect=65, shrink=1, pad=0.05, extendrect='True')
    cb.set_label('(m)', fontsize=12)
    
    ax = fig.add_subplot(2, 1, 2, projection=ccrs.PlateCarree())
    plt.title('Mean Sea Level Pressure', fontsize=10, loc='center')
    plot_background(ax)
    cf = ax.contourf(lons_mslp, lats_mslp, mslp_ana[i,:,:], levels=clevs2, cmap=cmap, extend='both',
                     transform=ccrs.PlateCarree())
    c = ax.contour(lons_mslp, lats_mslp, mslp_ana[i,:,:], levels=clevs2, colors='black', linewidths=0.5,
                   transform=ccrs.PlateCarree())
    ax.clabel(c, fmt='%4.1i', fontsize=10)
    plot_maxmin_points(lon_grid_mslp, lat_grid_mslp, mslp_ana_filter[i,:,:], 'max', 50, symbol='H', color='r',
                       transform=ccrs.PlateCarree())
    plot_maxmin_points(lon_grid_mslp, lat_grid_mslp, mslp_ana_filter[i,:,:], 'min', 25, symbol='L', color='b',
                       transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='vertical', aspect=65, shrink=1, pad=0.05, extendrect='True')
    cb.set_label('(hPa)', fontsize=12)
    
    plt.close()
    figname='./anim/Z500_MSLP_'+dates_str[i]
    fig.savefig(figname+'.png')
    
gif_filepath = './anim/Z500_MSLP.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

(21,)
2012-09-15T00
2012-09-15T06
2012-09-15T12
2012-09-15T18
2012-09-16T00
2012-09-16T06
2012-09-16T12
2012-09-16T18
2012-09-17T00
2012-09-17T06
2012-09-17T12
2012-09-17T18
2012-09-18T00
2012-09-18T06
2012-09-18T12
2012-09-18T18
2012-09-19T00
2012-09-19T06
2012-09-19T12
2012-09-19T18
2012-09-20T00


# Partie 2 : étude des données de prévision déterministe (20-30 septembre 2012)

In [106]:
z_fcst    = xr.open_dataset(Fcst_Z)
mslp_fcst    = xr.open_dataset(Fcst_MSLP)

print(z_fcst)
print(mslp_fcst)

<xarray.Dataset>
Dimensions:  (lon: 2400, lat: 1201, lev: 8, time: 21)
Coordinates:
  * lon      (lon) float64 0.0 0.15 0.3 0.45 0.6 ... 359.4 359.6 359.7 359.8
  * lat      (lat) float64 90.0 89.85 89.7 89.55 ... -89.55 -89.7 -89.85 -90.0
  * lev      (lev) float64 3e+04 4e+04 5e+04 6e+04 7e+04 8e+04 8.5e+04 9e+04
  * time     (time) datetime64[ns] 2012-09-20 2012-09-20T12:00:00 ... 2012-09-30
Data variables:
    var129   (time, lev, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version 1.4.3
    Conventions:  CF-1.0
    history:      Fri Nov 22 10:49:32 2019: cdo -f nc copy geop_fc_20120920-a...
    institution:  European Centre for Medium-Range Weather Forecasts
    CDO:          Climate Data Operators version 1.4.3 (http://www.mpimet.mpg...
<xarray.Dataset>
Dimensions:  (lon: 2560, lat: 1280, time: 21)
Coordinates:
  * lon      (lon) float64 0.0 0.1406 0.2812 0.4219 ... 359.4 359.6 359.7 359.9
  * lat      (lat) float64 89.89 89.75 89.61 89.47 ... -89.6

In [107]:
data_zf    = xr.open_dataset(Fcst_Z).sel(lev=50000).squeeze()
z_fcst=data_zf['var129']/9.81

data_mslpf    = xr.open_dataset(Fcst_MSLP)
mslp_fcst=data_mslpf['var151']/100

timef = data_zf.time.values
timesf_str=[x for x in range(len(timef))]
datesf_str=[x for x in range(len(timef))]

for i in range(len(timef)):
	timesf_str[i] = str(timef[i])
	datesf_str[i] = timesf_str[i][0:13]

print(datesf_str)

['2012-09-20T00', '2012-09-20T12', '2012-09-21T00', '2012-09-21T12', '2012-09-22T00', '2012-09-22T12', '2012-09-23T00', '2012-09-23T12', '2012-09-24T00', '2012-09-24T12', '2012-09-25T00', '2012-09-25T12', '2012-09-26T00', '2012-09-26T12', '2012-09-27T00', '2012-09-27T12', '2012-09-28T00', '2012-09-28T12', '2012-09-29T00', '2012-09-29T12', '2012-09-30T00']


In [108]:
lon_name = 'lon'

z_fcst['_longitude_adjusted'] = xr.where(
    z_fcst[lon_name] > 180,
    z_fcst[lon_name] - 360,
    z_fcst[lon_name])
z_fcst = (
    z_fcst
    .swap_dims({lon_name: '_longitude_adjusted'})
    .sel(**{'_longitude_adjusted': sorted(z_fcst._longitude_adjusted)})
    .drop(lon_name))
z_fcst = z_fcst.rename({'_longitude_adjusted': lon_name})

mslp_fcst['_longitude_adjusted'] = xr.where(
    mslp_fcst[lon_name] > 180,
    mslp_fcst[lon_name] - 360,
    mslp_fcst[lon_name])
mslp_fcst = (
    mslp_fcst
    .swap_dims({lon_name: '_longitude_adjusted'})
    .sel(**{'_longitude_adjusted': sorted(mslp_fcst._longitude_adjusted)})
    .drop(lon_name))
mslp_fcst = mslp_fcst.rename({'_longitude_adjusted': lon_name})

lats_zf = z_fcst.lat.values
lons_zf = z_fcst.lon.values
lats_mslpf = mslp_fcst.lat.values
lons_mslpf = mslp_fcst.lon.values

print(lats_zf)
print(lons_zf)

print(lats_mslpf)
print(lons_mslpf)

[ 90.    89.85  89.7  ... -89.7  -89.85 -90.  ]
[-179.85 -179.7  -179.55 ...  179.7   179.85  180.  ]
[ 89.89239645  89.75300494  89.61279026 ... -89.61279026 -89.75300494
 -89.89239645]
[-179.859375 -179.71875  -179.578125 ...  179.71875   179.859375
  180.      ]


In [109]:
z_fcst_filter = gaussian_filter(z_fcst, sigma=2.0)
mslp_fcst_filter = gaussian_filter(mslp_fcst, sigma=2.0)

lon_grid_zf, lat_grid_zf = np.meshgrid(lons_zf, lats_zf)
lon_grid_mslpf, lat_grid_mslpf = np.meshgrid(lons_mslpf, lats_mslpf)

In [110]:
cmap='jet'
clevs1 = np.linspace(5000, 6000, 21)

for i in range(len(timef)):
    print(datesf_str[i])
    fig = plt.figure(figsize=(15, 10))
    fig.suptitle('Analysis : '+datesf_str[0]+' - Forecast : '+datesf_str[i], fontsize=14)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    plt.title('Geopotential height at 500 hPa', size=10, loc='center')
    plot_background(ax)
    cf = ax.contourf(lons_zf, lats_zf, z_fcst[i,:,:], levels=clevs1, cmap=cmap, extend='both',
                     transform=ccrs.PlateCarree())
    c = ax.contour(lons_zf, lats_zf, z_fcst[i,:,:], levels=clevs1, colors='black', linewidths=0.5,
                   transform=ccrs.PlateCarree())
    ax.clabel(c, fmt='%4.1i', fontsize=10)
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.05, extendrect='True')
    cb.set_label('(m)', fontsize=12)
    plt.close()
    figname='./anim/Z500_'+datesf_str[i]
    fig.savefig(figname+'.png')
    
gif_filepath = './anim/Z500.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

2012-09-20T00
2012-09-20T12
2012-09-21T00
2012-09-21T12
2012-09-22T00
2012-09-22T12
2012-09-23T00
2012-09-23T12
2012-09-24T00
2012-09-24T12
2012-09-25T00
2012-09-25T12
2012-09-26T00
2012-09-26T12
2012-09-27T00
2012-09-27T12
2012-09-28T00
2012-09-28T12
2012-09-29T00
2012-09-29T12
2012-09-30T00


In [111]:
cmap='jet'
clevs2 = np.linspace(980, 1040, 21)
bounds = [(lonW, lonE, latS, latN)]

for i in range(len(timef)):
    print(datesf_str[i])
    fig = plt.figure(figsize=(15, 10))
    fig.suptitle('Analysis : '+datesf_str[0]+' - Forecast : '+datesf_str[i], fontsize=14)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    plt.title('Mean Sea Level Pressure', fontsize=10, loc='center')
    ax.coastlines()
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    cf = ax.contourf(lons_mslpf, lats_mslpf, mslp_fcst[i,:,:], levels=clevs2, cmap=cmap, extend='both',
                     transform=ccrs.PlateCarree())
    c = ax.contour(lons_mslpf, lats_mslpf, mslp_fcst[i,:,:], levels=clevs2, colors='black', linewidths=0.5,
                   transform=ccrs.PlateCarree())
    ax.clabel(c, fmt='%4.1i', fontsize=10)
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.05, extendrect='True')
    cb.set_label('(hPa)', fontsize=12)
    plt.close()
    figname='./anim/MSLP_'+datesf_str[i]
    fig.savefig(figname+'.png')

gif_filepath = './anim/MSLP.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

2012-09-20T00
2012-09-20T12
2012-09-21T00
2012-09-21T12
2012-09-22T00
2012-09-22T12
2012-09-23T00
2012-09-23T12
2012-09-24T00
2012-09-24T12
2012-09-25T00
2012-09-25T12
2012-09-26T00
2012-09-26T12
2012-09-27T00
2012-09-27T12
2012-09-28T00
2012-09-28T12
2012-09-29T00
2012-09-29T12
2012-09-30T00
